#Install Transformers

In [ ]:
!pip install datasets
!pip install -U sentence-transformers
!pip install transformers
!pip install rdflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=6b2514737db08f47a1a1f6a40b6b493997f043620b

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import libraries

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv
from datasets import load_dataset
import pandas as pd
import numpy as np

#Using FremyCompany/BioLORD-STAMB2-v1

In [ ]:
# Parameter Settings
model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+model_name

#Load BL Training Files

In [ ]:
#Training Data Files
train_samples = []
dev_samples = []
test_samples = []

# Dataset for training
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInp_train.csv", split="train")

for row in dataset:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  train_samples.append(inp_example)

# Dataset for validation
val = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInp_val.csv", split="train")

for row in val:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  dev_samples.append(inp_example)

# Dataset for test
test = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInp_test.csv", split="train")

for row in test:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  test_samples.append(inp_example)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

#FT : Fine Tune the model, Training Files = BL

---



In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+base_model_name+'FT'

In [ ]:
#Define the model. Either from scratch of by loading a pre-trained model
modelFT = SentenceTransformer(base_model_name)

In [ ]:
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name+'FT'

#evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples)

#Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=modelFT)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

#Tune the model
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
modelFT.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

Iteration:   0%|          | 0/537 [00:00<?, ?it/s]

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

modelFT = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples)
test_evaluator(modelFT, output_path=model_save_path)

0.9154633382222863

In [ ]:
model_orig = SentenceTransformer(base_model_name)
test_evaluator(model_orig)

0.7820252553793126

#Load KS Training Files

In [ ]:
#Training Data Files
trainSym_samples = []
devSym_samples = []
testSym_samples = []

# Dataset for training
datasetSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_train.csv", split="train")

for row in datasetSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  trainSym_samples.append(inp_example)

# Dataset for validation
valSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_val.csv", split="train")

for row in valSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  devSym_samples.append(inp_example)

# Dataset for test
testSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_test.csv", split="train")

for row in testSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  testSym_samples.append(inp_example)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

# FS : Fine Tune the Model, Training Files = KS

In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+base_model_name + 'FS'

In [ ]:
#Define the model. Either from scratch of by loading a pre-trained model
modelFS = SentenceTransformer(base_model_name)

In [ ]:
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FS'

#evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(devSym_samples)

#Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(trainSym_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=modelFS)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

#Tune the model
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
modelFS.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

modelFS = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(testSym_samples)
test_evaluator(modelFS, output_path=model_save_path)

0.7011037029911785

In [ ]:
model_orig = SentenceTransformer(base_model_name)
test_evaluator(model_orig)

0.795570655194067

#FDKD

In [ ]:
dfTrain = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_train.csv")
dfTrainClean = pd.DataFrame()
dfTrainClean["sentence1"] = dfTrain["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
dfTrainClean["sentence2"] = dfTrain["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
dfTrainClean["similarity_score"] = dfTrain["similarity_score"]
dfTrainClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKD_train.csv",index=False)

dfVal = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_val.csv")
dfValClean = pd.DataFrame()
dfValClean["sentence1"] = dfVal["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
dfValClean["sentence2"] = dfVal["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
dfValClean["similarity_score"] = dfVal["similarity_score"]
dfValClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKD_val.csv",index=False)

dfTest = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_test.csv")
dfTestClean = pd.DataFrame()
dfTestClean["sentence1"] = dfTest["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
dfTestClean["sentence2"] = dfTest["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))).replace(" :",""))
dfTestClean["similarity_score"] = dfTest["similarity_score"]
dfTestClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKD_test.csv",index=False)

dfTrain, dfTrainClean, dfVal, dfValClean, dfTest, dfTestClean

(                                              sentence1  \
 0     combined oxidative phosphorylation deficiency ...   
 1     combined oxidative phosphorylation deficiency ...   
 2                            basilar impression primary   
 3                      gaucher disease perinatal lethal   
 4                      gaucher disease perinatal lethal   
 ...                                                 ...   
 5242                                        craniorhiny   
 5243                                        craniorhiny   
 5244                                        craniorhiny   
 5245                                        craniorhiny   
 5246                                        craniorhiny   
 
                                               sentence2  similarity_score  \
 0     mitochondrial disorder due to a defect in mito...               1.0   
 1     mitochondrial substrate carrier disorder : neo...               1.0   
 2       medullar disease : primary basilar 

In [ ]:
#FD Training Data Files
trainFD_samples = []
devFD_samples = []
testFD_samples = []

# Dataset for training
datasetSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKD_train.csv", split="train")

for row in datasetSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  trainFD_samples.append(inp_example)

# Dataset for validation
valSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKD_val.csv", split="train")

for row in valSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  devFD_samples.append(inp_example)

# Dataset for test
testSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKD_test.csv", split="train")

for row in testSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  testFD_samples.append(inp_example)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(datasetSym)

Dataset({
    features: ['sentence1', 'sentence2', 'similarity_score'],
    num_rows: 4223
})


In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FD'

#Define the model. Either from scratch of by loading a pre-trained model
modelFD = SentenceTransformer(base_model_name)

#evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(devFD_samples)

#Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(trainFD_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=modelFD)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

#Tune the model
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
modelFD.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

modelFD = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(testFD_samples)
test_evaluator(modelFD, output_path=model_save_path)

0.7396021905444724

In [ ]:
model_orig = SentenceTransformer(base_model_name)
test_evaluator(model_orig)

0.7874947142675051

#FCKC

Fine Tune Model = FC,
Training Files = KS (KS + colon replaced with comma),
OntFiles = (KS + colon replaced with comma)

In [ ]:
#Training Data Files = KS (KS + colon replaced with comma)
trainFC_samples = []
devFC_samples = []
testFC_samples = []

# Dataset for training
datasetSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_train.csv", split="train")

for row in datasetSym:
  #print(row['sentence1'])
  row['sentence1'] = row['sentence1'].replace(":",",")
  row['sentence2'] = row['sentence2'].replace(":",",")
  #print(row['sentence1'])
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  trainFC_samples.append(inp_example)

# Dataset for validation
valSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_val.csv", split="train")

for row in valSym:
  row['sentence1'] = row['sentence1'].replace(":",",")
  row['sentence2'] = row['sentence2'].replace(":",",")
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  devFC_samples.append(inp_example)

# Dataset for test
testSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_test.csv", split="train")

for row in testSym:
  row['sentence1'] = row['sentence1'].replace(":",",")
  row['sentence2'] = row['sentence2'].replace(":",",")
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  testFC_samples.append(inp_example)

In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+base_model_name+'FT'

In [ ]:
modelFC = SentenceTransformer(base_model_name)

In [ ]:
#Fine tune the model = FC

model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FC'

#evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(devFC_samples)

#Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(trainFC_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=modelFC)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

#Tune the model
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
modelFC.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

modelFC = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(testFC_samples)
test_evaluator(modelFC, output_path=model_save_path)

0.45860737932707935

In [ ]:
model_orig = SentenceTransformer(base_model_name)
test_evaluator(model_orig)

0.7958489274639798

#FSDKSD

(KI with sym colon but no duplicates)

In [ ]:
dfTrain = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_train.csv")
dfTrainClean = pd.DataFrame()
dfTrainClean["sentence1"] = dfTrain["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfTrainClean["sentence2"] = dfTrain["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfTrainClean["similarity_score"] = dfTrain["similarity_score"]
dfTrainClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKSD_train.csv",index=False)

dfVal = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_val.csv")
dfValClean = pd.DataFrame()
dfValClean["sentence1"] = dfVal["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfValClean["sentence2"] = dfVal["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfValClean["similarity_score"] = dfVal["similarity_score"]
dfValClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKSD_val.csv",index=False)

dfTest = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_test.csv")
dfTestClean = pd.DataFrame()
dfTestClean["sentence1"] = dfTest["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfTestClean["sentence2"] = dfTest["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfTestClean["similarity_score"] = dfTest["similarity_score"]
dfTestClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKSD_test.csv",index=False)

dfTrain, dfTrainClean, dfVal, dfValClean, dfTest, dfTestClean

(                                              sentence1  \
 0     combined oxidative phosphorylation deficiency ...   
 1     combined oxidative phosphorylation deficiency ...   
 2                            basilar impression primary   
 3                      gaucher disease perinatal lethal   
 4                      gaucher disease perinatal lethal   
 ...                                                 ...   
 5242                                        craniorhiny   
 5243                                        craniorhiny   
 5244                                        craniorhiny   
 5245                                        craniorhiny   
 5246                                        craniorhiny   
 
                                               sentence2  similarity_score  \
 0     mitochondrial disorder due to a defect in mito...               1.0   
 1     mitochondrial substrate carrier disorder : neo...               1.0   
 2       medullar disease : primary basilar 

In [ ]:
#Training Data Files
trainFSD_samples = []
devFSD_samples = []
testFSD_samples = []

# Dataset for training
datasetSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKSD_train.csv", split="train")

for row in datasetSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  trainFSD_samples.append(inp_example)

# Dataset for validation
valSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKSD_val.csv", split="train")

for row in valSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  devFSD_samples.append(inp_example)

# Dataset for test
testSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKSD_test.csv", split="train")

for row in testSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  testFSD_samples.append(inp_example)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#model_save_path = '/content/drive/MyDrive/Thesis/output/finetuneSbert-'+base_model_name+'FT'

In [ ]:
modelFSD = SentenceTransformer(base_model_name)

In [ ]:
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FSD'

#evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(devFSD_samples)

#Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(trainFSD_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=modelFSD)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

#Tune the model
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
modelFSD.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

modelFSD = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(testFSD_samples)
test_evaluator(modelFSD, output_path=model_save_path)

0.7061548479142329

In [ ]:
model_orig = SentenceTransformer(base_model_name)
test_evaluator(model_orig)

0.792008706873236

#FIKI

In [ ]:
#Training Data Files = KI (KS minus colon)
trainFI_samples = []
devFI_samples = []
testFI_samples = []

# Dataset for training
datasetSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_train.csv", split="train")

for row in datasetSym:
  #print(row['sentence1'])
  row['sentence1'] = row['sentence1'].replace(":","")
  row['sentence2'] = row['sentence2'].replace(":","")
  #print(row['sentence1'])
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  trainFI_samples.append(inp_example)

# Dataset for validation
valSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_val.csv", split="train")

for row in valSym:
  row['sentence1'] = row['sentence1'].replace(":","")
  row['sentence2'] = row['sentence2'].replace(":","")
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  devFI_samples.append(inp_example)

# Dataset for test
testSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpSym_test.csv", split="train")

for row in testSym:
  row['sentence1'] = row['sentence1'].replace(":","")
  row['sentence2'] = row['sentence2'].replace(":","")
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  testFI_samples.append(inp_example)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
#model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name+'FT'

In [ ]:
modelFI = SentenceTransformer(base_model_name)

In [ ]:
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FI'

#evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(devFI_samples)

#Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(trainFI_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=modelFI)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

#Tune the model
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
modelFI.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

Iteration:   0%|          | 0/328 [00:00<?, ?it/s]

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

modelFI = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(testFI_samples)
test_evaluator(modelFI, output_path=model_save_path)

0.7204698118147912

In [ ]:
model_orig = SentenceTransformer(base_model_name)
test_evaluator(model_orig)

0.7932011219777931

#FJKJ

In [ ]:
#Training Data Files = KI (KS minus colon)
trainFJ_samples = []
devFJ_samples = []
testFJ_samples = []

# Dataset for training
datasetSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpJoin_train.csv", split="train")

for row in datasetSym:
  #print(row['sentence1'])
  #print(row['sentence1'])
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  trainFJ_samples.append(inp_example)

# Dataset for validation
valSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpJoin_val.csv", split="train")

for row in valSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  devFJ_samples.append(inp_example)

# Dataset for test
testSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpJoin_test.csv", split="train")

for row in testSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  testFJ_samples.append(inp_example)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
#model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name+'FT'

In [ ]:
modelFJ = SentenceTransformer(base_model_name)

In [ ]:
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FJ'

#evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(devFJ_samples)

#Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(trainFJ_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=modelFJ)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

#Tune the model
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
modelFJ.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

modelFJ = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(testFJ_samples)
test_evaluator(modelFJ, output_path=model_save_path)

0.815112068794951

In [ ]:
model_orig = SentenceTransformer(base_model_name)
test_evaluator(model_orig)

0.7934969994364606

#FJDKJD

In [ ]:
dfTrain = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpJoin_train.csv")
dfTrainClean = pd.DataFrame()
dfTrainClean["sentence1"] = dfTrain["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfTrainClean["sentence2"] = dfTrain["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfTrainClean["similarity_score"] = dfTrain["similarity_score"]
dfTrainClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKJD_train.csv",index=False)

dfVal = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpJoin_val.csv")
dfValClean = pd.DataFrame()
dfValClean["sentence1"] = dfVal["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfValClean["sentence2"] = dfVal["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfValClean["similarity_score"] = dfVal["similarity_score"]
dfValClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKJD_val.csv",index=False)

dfTest = pd.read_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpJoin_test.csv")
dfTestClean = pd.DataFrame()
dfTestClean["sentence1"] = dfTest["sentence1"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfTestClean["sentence2"] = dfTest["sentence2"].apply(lambda row: (' '.join(dict.fromkeys(row.split()))))
dfTestClean["similarity_score"] = dfTest["similarity_score"]
dfTestClean.to_csv("/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKJD_test.csv",index=False)

dfTrain, dfTrainClean, dfVal, dfValClean, dfTest, dfTestClean

(                                              sentence1  \
 0     combined oxidative phosphorylation deficiency ...   
 1                            basilar impression primary   
 2                      gaucher disease perinatal lethal   
 3      pulmonary atresia with intact ventricular septum   
 4     ceroid lipofuscinoses : ceroid lipofuscinosis ...   
 ...                                                 ...   
 2228            martsolf syndrome : martsolf syndrome 1   
 2229            martsolf syndrome : martsolf syndrome 1   
 2230            martsolf syndrome : martsolf syndrome 1   
 2231            martsolf syndrome : martsolf syndrome 1   
 2232            martsolf syndrome : martsolf syndrome 1   
 
                                               sentence2  similarity_score  \
 0     mitochondrial substrate carrier disorder mitoc...               1.0   
 1       medullar disease : primary basilar invagination               1.0   
 2     autosomal ichthyosis syndrome with fa

In [ ]:
#Training Data Files
trainFJD_samples = []
devFJD_samples = []
testFJD_samples = []

# Dataset for training
datasetSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKJD_train.csv", split="train")

for row in datasetSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  trainFJD_samples.append(inp_example)

# Dataset for validation
valSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKJD_val.csv", split="train")

for row in valSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  devFJD_samples.append(inp_example)

# Dataset for test
testSym = load_dataset("csv", data_files="/content/drive/MyDrive/Thesis/OMIM2ORDO/bertInpKJD_test.csv", split="train")

for row in testSym:
  inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['similarity_score'])
  testFJD_samples.append(inp_example)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Parameter Settings
base_model_name = 'FremyCompany/BioLORD-STAMB2-v1'
train_batch_size = 16
num_epochs = 15
#model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
#model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name+'FT'

In [ ]:
modelFJD = SentenceTransformer(base_model_name)

In [ ]:
model_save_path = '/content/drive/MyDrive/Thesis/OMIM2ORDO/output/finetuneSbert-'+base_model_name +'FJD'

#evaluator
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(devFJD_samples)

#Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(trainFJD_samples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model=modelFJD)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

#Tune the model
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
#model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
modelFJD.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140 [00:00<?, ?it/s]

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

modelFJD = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(testFJD_samples)
test_evaluator(modelFJD, output_path=model_save_path)

0.8176569284428836

In [ ]:
model_orig = SentenceTransformer(base_model_name)
test_evaluator(model_orig)

0.7885216847530163